<a href="https://colab.research.google.com/github/samibahig/IFT6390/blob/main/kaggle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import scipy.sparse as sp 
import pandas as pd 
# import sklearn.utils.extmath.safe_sparse_dot
import gc
import re, unicodedata
import string
import tqdm
from collections import defaultdict

In [ ]:
train, test = pd.read_csv('/content/train.csv'), pd.read_csv('/content/test.csv')
print (train, test)

        Id  ...           Category
0        0  ...           astro-ph
1        1  ...             hep-ph
2        2  ...              cs.LG
3        3  ...            math.CO
4        4  ...  cond-mat.mes-hall
...    ...  ...                ...
7495  7495  ...        astro-ph.CO
7496  7496  ...        astro-ph.CO
7497  7497  ...             hep-th
7498  7498  ...            math.CO
7499  7499  ...             hep-th

[7500 rows x 3 columns]           Id                                           Abstract
0          0    We describe ways to define and calculate $L_...
1          1    The progenitor systems of Type-Ia supernovae...
2          2    OmegaWhite is a wide-field, high cadence, sy...
3          3    Given $n \geq 2$ and $1<p<n$, we consider th...
4          4    The motivation of this work is to improve th...
...      ...                                                ...
14995  14995    We investigate the ability of the Space Infr...
14996  14996    We study theoretically the 

In [ ]:
# print(train.keys())

print(train['Abstract'])

0         The energy released in a solar flare is part...
1         In light of current atmospheric neutrino osc...
2         We consider the following basic learning tas...
3         In this paper, we characterise the family of...
4         The control of condensed matter systems out ...
                              ...                        
7495      We investigate the dynamical behaviour of a ...
7496      We use the full bispectrum of spherical need...
7497      We show that Ostrogradsky ghosts in higher-d...
7498      McMorris and Powers proved an Arrow-type the...
7499      Recently it has been speculated that the War...
Name: Abstract, Length: 7500, dtype: object


In [ ]:
def remove_non_ascii(words):
    return ''.join([i if ord(i) < 128 else ' ' for i in words])

In [ ]:
def process(df, t):
    df[t] = df[t].apply(lambda x : x.lower())
    #train['Abstract'] = train['Abstract'].apply(lambda x : remove_punctuation(x))
    df[t] = df[t].apply(lambda x : x.strip())
    df[t] = df[t].apply(lambda x : re.sub('\n', ' ', x))
    df[t] = df[t].apply(lambda x : re.sub('\[[^]]*\]', '', x))
    df[t] = df[t].apply(lambda x : re.sub("<.*?>", " ", x))
    df[t] = df[t].apply(lambda x : remove_non_ascii(x))
    df[t] = df[t].str.replace('[^\w\s]','')
    return df

In [ ]:
t = train['Abstract']

train = process(train, 'Abstract')
test = process(test, 'Abstract')

train['Abstract']
print(train['Abstract'].shape)

(7500,)


In [ ]:
class BernoulliVectorizer:
    def __init__(self):
        self.vocab = []
        self.vocab_counter = {}
    def build_vocab(self, data):
        for document in data:
            for word in document.split(' '):
                if word in self.vocab_counter:
                    self.vocab_counter[word] += 1
                    if self.vocab_counter[word] == 2:
                        self.vocab.append(word)
                else:
                    self.vocab_counter[word] = 1
    def transform(self, data):
        i = 0
        counter = 0 
        answer = np.zeros((len(data),len(self.vocab)))
        for document in data:
            counter = counter + 1
            token = document.split(' ')                                 
            bin_vect = np.zeros(len(self.vocab))
            for word_idx in range(len(self.vocab)):
                for e in token: 
                    if e == self.vocab[word_idx]:
                        bin_vect[word_idx ] = 1
            answer[i, :] = bin_vect
            i += 1
        return answer
    
    def fit_transform(self, data):
        self.build_vocab(data)
        print(len(self.vocab))
        return self.transform(data)

In [ ]:
BV = BernoulliVectorizer()
#X_train = BV.fit_transform(train['Abstract'])
#test['Abstract'] = B.transform(test['Abstract'])
# train.head()
# train  = train.to_numpy() 

In [ ]:
class BernoulliNB:
    def __init__(self, alpha):
        self.alpha = alpha   
    '''P(C_k) =  number of documents of that class / number of documents
       P(C_k / w1, w2, ...)\prop P(C_k) P(w1/C_k) P(w2/ C_k)
       P(wi / C_k) = number of documents of class C_k with w_i/ number of documents with that class 
       get all the rows of class C_k, how many of them was word w_i 
       get all the rows of class C_k, how many of them has a 1 in the index of w_i '''
    def fit(self, X, y):
        self.n_classes = len(np.unique(np.unique(y,return_inverse=True)[1]))
        n_classes = self.n_classes
        # calculate P(C_k) for all k
        self.counts = np.zeros(n_classes)
        y_cat = np.unique(y,return_inverse=True)[1]
        for i in y_cat:
            self.counts[i] += 1,
        self.counts /= len(y_cat)
     # generate n_features x n_classes matrix\n",   
        self.params = np.zeros((n_classes, X.shape[1]))
        for idx in range(len(X)):
            self.params[y_cat[idx]] += X[idx]
        self.params += self.alpha 
        class_sums = self.params.sum(axis=1) +  self.alpha * self.n_classes  #laplace
        self.params = self.params / class_sums[:, np.newaxis]
    def predict(self, X):
        neg_prob = np.log(1 - self.params)
        # compute neg_prob
        jll = np.dot(X, (np.log(self.params)-neg_prob).T)           
        jll += np.log(self.counts) + neg_prob.sum(axis=1)
        return np.argmax(jll, axis=1)

In [ ]:
uni, cats = np.unique(train['Category'],return_inverse=True)

In [ ]:
alpha = 1
NB = BernoulliNB(alpha)
X_train = BV.fit_transform(train['Abstract'])
NB.fit(X_train, train['Category'])
print('Alhumdulillah')
X_test = BV.transform(test['Abstract'])
Y_test = NB.predict(X_test)


21485
Alhumdulillah
Alhumdulillah


In [ ]:
Y_submit = uni[Y_test]

In [ ]:
Y_predict = NB.predict(X_train)


In [ ]:
np.mean(cats == Y_predict)

0.9297333333333333

In [ ]:
pd.DataFrame(Y_submit).to_csv('Bernoulli21485.csv')

In [ ]:
def remove_punctuation(text):
    pass

def remove_non_ascii(words):
    return ''.join([i if ord(i) < 128 else ' ' for i in words])
#    new_words = []
#    for word in words:
#        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#        new_words.append(new_word)
#        return ''.join(new_words)
def process(df, t):
    df[t] = df[t].apply(lambda x : x.lower())
    #train['Abstract'] = train['Abstract'].apply(lambda x : remove_punctuation(x))
    df[t] = df[t].apply(lambda x : x.strip())
    df[t] = df[t].apply(lambda x : re.sub('\n', ' ', x))
    df[t] = df[t].apply(lambda x : re.sub('\[[^]]*\]', '', x))
    df[t] = df[t].apply(lambda x : re.sub("<.*?>", " ", x))
    df[t] = df[t].apply(lambda x : remove_non_ascii(x))
    df[t] = df[t].str.replace('[^\w\s]','')
    print(df.head())
    return df
    #train['Abstract'] = train['Abstract'].apply(lambda x: remove_punctuation(x))

In [ ]:
Features_train = train['Abstract']
Labels_train = train['Category']

In [ ]:
train = process(train, 'Abstract')
test = process(test, 'Abstract')

   Id                                           Abstract           Category
0   0  the energy released in a solar flare is partit...           astro-ph
1   1  in light of current atmospheric neutrino oscil...             hep-ph
2   2  we consider the following basic learning task ...              cs.LG
3   3  in this paper we characterise the family of fi...            math.CO
4   4  the control of condensed matter systems out of...  cond-mat.mes-hall
   Id                                           Abstract
0   0  we describe ways to define and calculate l_1no...
1   1  the progenitor systems of typeia supernovae sn...
2   2  omegawhite is a widefield high cadence synopti...
3   3  given n geq 2 and 1pn we consider the critical...
4   4  the motivation of this work is to improve the ...


In [ ]:
train['Abstract']
print(train['Abstract'].shape)

(7500,)


In [ ]:
class BernoulliVectorizer:
    def __init__(self):
        self.vocab = []
        self.vocab_counter = {}
    def build_vocab(self, data):
        for document in data:
            for word in document.split(' '):
                if word in self.vocab_counter:
                    self.vocab_counter[word] += 1
                    if self.vocab_counter[word] == 500:
                        self.vocab.append(word)
                else:
                    self.vocab_counter[word] = 1
    def transform(self, data):
        i = 0
        #counter = 0 
        answer = np.zeros((len(data),len(self.vocab)))
        for document in data:
            #counter = counter + 1
            token = document.split(' ')                                 
            bin_vect = np.zeros(len(self.vocab))
            for word_idx in range(len(self.vocab)):
                for e in token: 
                    if e == self.vocab[word_idx]:
                        bin_vect[word_idx ] = 1
            answer[i, :] = bin_vect
            i += 1
        return answer
    
    def fit_transform(self, data):
        self.build_vocab(data)
        print(len(self.vocab))
        return self.transform(data)

In [ ]:
class BernoulliNB:
    def __init__(self, alpha):
        self.alpha = alpha   
    '''P(C_k) =  number of documents of that class / number of documents
       P(C_k / w1, w2, ...)\prop P(C_k) P(w1/C_k) P(w2/ C_k)
       P(wi / C_k) = number of documents of class C_k with w_i/ number of documents with that class 
       get all the rows of class C_k, how many of them was word w_i 
       get all the rows of class C_k, how many of them has a 1 in the index of w_i '''
    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        n_classes = self.n_classes
        # calculate P(C_k) for all k)
        y_cat = np.unique(y,return_inverse=True)[1]
        self.counts = np.zeros(n_classes)
        for i in y_cat:
            self.counts[i] += 1,
        self.counts /= len(y_cat)
     # generate n_features x n_classes matrix\n",   
        self.params = np.zeros((n_classes, X.shape[1])),
        for idx in range(len(X)):
            self.params[y_cat[idx]] += X[idx]
        self.params += self.alpha 
        class_sums = self.params.sum(axis=1) +  self.alpha * self.n_classes  #laplace
        self.params = self.params / class_sums[:, np.newaxis]
    def predict(self, X):
        neg_prob = np.log(1 - self.params)
        # compute neg_prob
        jll = np.dot(X, (np.log(self.params)-neg_prob).T)           
        jll += np.log(self.counts) + neg_prob.sum(axis=1)
        return np.argmax(jll, axis=1)

In [ ]:
B = BernoulliVectorizer()
alpha = 1
NB = BernoulliNB(alpha)
X_train = B.fit_transform(train['Abstract'])
X_test = B.transform(test['Abstract'])
NB.fit(X_train, train["Category"])
predictions = NB.predict(X_test)
train_predictions= NB.predict(X_train)

21485


KeyboardInterrupt: ignored